In [ ]:
import hsfs
from pyspark.sql.types import *

In [ ]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

In [ ]:
df = spark.read.format("net.snowflake.spark.snowflake") \
  .options(**sfOptions) \
  .load()

df.show(10)